In [1]:
import scanpy as sc
import anndata
import pandas as pd
import os

In [2]:
batch_id = '151673'
input_path = os.path.join('data/st_data/DLPFC_new', batch_id)
adata = sc.read_visium(input_path, count_file='filtered_feature_bc_matrix.h5')

/home/lytq/Hist2ST/hist2st_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/lytq/Hist2ST/hist2st_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata.var_names_make_unique()

In [4]:
adata

AnnData object with n_obs × n_vars = 3639 × 33538
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [5]:
adata_temp = sc.read_10x_h5(os.path.join(input_path, 'filtered_feature_bc_matrix.h5'))

/home/lytq/Hist2ST/hist2st_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/lytq/Hist2ST/hist2st_env/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
adata_temp

AnnData object with n_obs × n_vars = 3639 × 33538
    var: 'gene_ids', 'feature_types', 'genome'

In [7]:
img_path = os.path.join(input_path, 'spatial', 'tissue_hires_image.png')

In [8]:
metadata_path = os.path.join(input_path, 'metadata.tsv')
positions_path = os.path.join(input_path, 'spatial', 'tissue_positions_list.csv')

# Read metadata
metadata = pd.read_csv(metadata_path, sep='\t')
spatial_positions = pd.read_csv(positions_path, header=None)

In [14]:
adata.obs['layer_guess']

AAACAAGTATCTCCCA-1    Layer3
AAACAATCTACTAGCA-1    Layer1
AAACACCAATAACTGC-1        WM
AAACAGAGCGACTCCT-1    Layer3
AAACAGCTTTCAGAAG-1    Layer5
                       ...  
TTGTTTCACATCCAGG-1        WM
TTGTTTCATTAGTCTA-1        WM
TTGTTTCCATACAACT-1    Layer6
TTGTTTGTATTACACG-1        WM
TTGTTTGTGTAAATTC-1    Layer2
Name: layer_guess, Length: 3639, dtype: object

In [15]:
adata.layers['count'] = adata.X.toarray()

In [16]:
adata

AnnData object with n_obs × n_vars = 3639 × 33538
    obs: 'barcode', 'sample_name', 'tissue', 'row', 'col', 'imagerow', 'imagecol', 'Cluster', 'height', 'width', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'subject_position', 'discard', 'key', 'cell_count', 'SNN_k50_k4', 'SNN_k50_k5', 'SNN_k50_k6', 'SNN_k50_k7', 'SNN_k50_k8', 'SNN_k50_k9', 'SNN_k50_k10', 'SNN_k50_k11', 'SNN_k50_k12', 'SNN_k50_k13', 'SNN_k50_k14', 'SNN_k50_k15', 'SNN_k50_k16', 'SNN_k50_k17', 'SNN_k50_k18', 'SNN_k50_k19', 'SNN_k50_k20', 'SNN_k50_k21', 'SNN_k50_k22', 'SNN_k50_k23', 'SNN_k50_k24', 'SNN_k50_k25', 'SNN_k50_k26', 'SNN_k50_k27', 'SNN_k50_k28', 'GraphBased', 'Maynard', 'Martinowich', 'Layer', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'expr_chrM', 'expr_chrM_ratio', 'SpatialDE_PCA', 'SpatialDE_pool_PCA', 'HVG_PCA', 'pseudobulk_PCA', 'markers_PCA', 'SpatialDE_UMAP', 'SpatialDE_pool_UMAP', 'HVG_UMAP', 'pseudobulk_UMAP', 'markers_UMAP', 'SpatialDE_PCA_spatial', 'SpatialD

In [17]:
sc.pp.filter_genes(adata, min_cells=50)
sc.pp.filter_genes(adata, min_counts=10)
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", layer='count', n_top_genes=1000)
# sc.pp.log1p(adata)


from sklearn.decomposition import PCA  # sklearn PCA is used because PCA in scanpy is not stable.
adata_X = PCA(n_components=200, random_state=42).fit_transform(adata.X)
adata.obsm['X_pca'] = adata_X

In [18]:
adata

AnnData object with n_obs × n_vars = 3639 × 13104
    obs: 'barcode', 'sample_name', 'tissue', 'row', 'col', 'imagerow', 'imagecol', 'Cluster', 'height', 'width', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'subject_position', 'discard', 'key', 'cell_count', 'SNN_k50_k4', 'SNN_k50_k5', 'SNN_k50_k6', 'SNN_k50_k7', 'SNN_k50_k8', 'SNN_k50_k9', 'SNN_k50_k10', 'SNN_k50_k11', 'SNN_k50_k12', 'SNN_k50_k13', 'SNN_k50_k14', 'SNN_k50_k15', 'SNN_k50_k16', 'SNN_k50_k17', 'SNN_k50_k18', 'SNN_k50_k19', 'SNN_k50_k20', 'SNN_k50_k21', 'SNN_k50_k22', 'SNN_k50_k23', 'SNN_k50_k24', 'SNN_k50_k25', 'SNN_k50_k26', 'SNN_k50_k27', 'SNN_k50_k28', 'GraphBased', 'Maynard', 'Martinowich', 'Layer', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'expr_chrM', 'expr_chrM_ratio', 'SpatialDE_PCA', 'SpatialDE_pool_PCA', 'HVG_PCA', 'pseudobulk_PCA', 'markers_PCA', 'SpatialDE_UMAP', 'SpatialDE_pool_UMAP', 'HVG_UMAP', 'pseudobulk_UMAP', 'markers_UMAP', 'SpatialDE_PCA_spatial', 'SpatialD

In [19]:
import torch
from HIST2ST import Hist2ST

import random
import numpy as np

[easydl] tensorflow not available!


In [20]:
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
# !pip uninstall -y pytorch-lightning
# !pip install pytorch-lightning==2.0.2
# import pytorch_lightning as pl
# print(pl.__version__)

In [21]:
device='cuda:1'
genes = adata.shape[1] # number of genes
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

model=Hist2ST(
    depth1=d1, depth2=d2, depth3=d3,
    n_genes=785, 
    kernel_size=k, patch_size=p,
    fig_size=112,
    learning_rate=1e-5,
    heads=h, channel=c, dropout=dropout,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
)

In [22]:
model.load_state_dict(torch.load(f'./models/5-Hist2ST.ckpt'))    

<All keys matched successfully>

In [23]:
from predict import pk_load, test
from graph_construction import calcADJ

In [24]:
adj_matrix = calcADJ(adata.obsm['X_pca'], 4)

In [25]:
len(adj_matrix)
type(adj_matrix)

torch.Tensor

In [26]:
adata.uns['adj'] = adj_matrix

In [27]:
adata

AnnData object with n_obs × n_vars = 3639 × 13104
    obs: 'barcode', 'sample_name', 'tissue', 'row', 'col', 'imagerow', 'imagecol', 'Cluster', 'height', 'width', 'sum_umi', 'sum_gene', 'subject', 'position', 'replicate', 'subject_position', 'discard', 'key', 'cell_count', 'SNN_k50_k4', 'SNN_k50_k5', 'SNN_k50_k6', 'SNN_k50_k7', 'SNN_k50_k8', 'SNN_k50_k9', 'SNN_k50_k10', 'SNN_k50_k11', 'SNN_k50_k12', 'SNN_k50_k13', 'SNN_k50_k14', 'SNN_k50_k15', 'SNN_k50_k16', 'SNN_k50_k17', 'SNN_k50_k18', 'SNN_k50_k19', 'SNN_k50_k20', 'SNN_k50_k21', 'SNN_k50_k22', 'SNN_k50_k23', 'SNN_k50_k24', 'SNN_k50_k25', 'SNN_k50_k26', 'SNN_k50_k27', 'SNN_k50_k28', 'GraphBased', 'Maynard', 'Martinowich', 'Layer', 'layer_guess', 'layer_guess_reordered', 'layer_guess_reordered_short', 'expr_chrM', 'expr_chrM_ratio', 'SpatialDE_PCA', 'SpatialDE_pool_PCA', 'HVG_PCA', 'pseudobulk_PCA', 'markers_PCA', 'SpatialDE_UMAP', 'SpatialDE_pool_UMAP', 'HVG_UMAP', 'pseudobulk_UMAP', 'markers_UMAP', 'SpatialDE_PCA_spatial', 'SpatialD

In [30]:
fold=8
testset = pk_load(fold,'test',dataset='dlpfc',flatten=False,adj=True,ori=True)

Loading imgs...
Loading metadata...


KeyError: "None of [Index(['KLHL17', 'PRDM16', 'C1orf174', 'GPR153', 'PER3', 'CLSTN1', 'NMNAT1',\n       'CORT', 'DISP3', 'MAD2L2',\n       ...\n       'SCAF4', 'CFAP298', 'OLIG2', 'OLIG1', 'CBR1', 'PCP4', 'TFF3', 'TFF1',\n       'S100B', 'MT-CO3'],\n      dtype='object', length=1000)] are in the [columns]"

In [ ]:
# Load patches of image and plot
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
from torchvision import transforms
from skimage import io

def load_image_patches(image_path, positions, patch_size=224):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((patch_size, patch_size)),
        transforms.ToTensor(),
    ])
    
    patches = []
    for _, row in positions.iterrows():
        x, y = int(row['image_x']), int(row['image_y'])
        patch = image.crop((x, y, x + patch_size, y + patch_size))
        patch_tensor = transform(patch)
        patches.append(patch_tensor)
    
    return patches

image_path = 'data/st_data/DLPFC_new/151673/spatial/tissue_hires_image.png'
patches = load_image_patches(image_path, spatial_positions)
# Plot the patches
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    if i < len(patches):
        ax.imshow(patches[i].permute(1, 2, 0))
    ax.axis('off')
plt.show()


KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader

test_loader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)
pred, gt = test(model, test_loader, device=device)

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: The expanded size of the tensor (112) must match the existing size (0) at non-singleton dimension 2.  Target sizes: [3, 112, 112].  Tensor sizes: [3, 0, 0]